In [2]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
plt.style.use('bmh')
from pathlib import Path
import pycountry
from pprint import pprint
import yaml
from copy import deepcopy

print(os.getcwd())

c:\Users\s2216495\Desktop\pypsa\my_pypsa_eur_sec\pypsa-eur-sec


In [7]:
pypsa_eur_path = Path.cwd() / '..' / '..' / 'lab_pypsa_eur_sec' / 'pypsa-eur'
respath = pypsa_eur_path / 'resources' 
egspath = Path.cwd() / "lukas_data"

with open(pypsa_eur_path / 'config.default.yaml', 'r') as f:
    config = yaml.safe_load(f)
    
print(respath)

c:\Users\s2216495\Desktop\pypsa\my_pypsa_eur_sec\pypsa-eur-sec\..\..\lab_pypsa_eur_sec\pypsa-eur\resources


In [9]:
pypsa_countries = gpd.read_file(respath / 'country_shapes.geojson')
pypsa_countries["full_names"] = pypsa_countries.name.apply(
    lambda name: pycountry.countries.get(alpha_2=name).name)

countryname_mapper = {
    'Macedonia': 'North Macedonia',
    'Bosnia and Herz.': 'Bosnia and Herzegovina',
    'Czech Republic': 'Czechia',
}

for i, row in pypsa_countries.iterrows():
    if row.full_names in countryname_mapper:
        pypsa_countries.at[i, 'full_names'] = countryname_mapper[row.full_names]


def load_egs_data(egs_path, shape_path):
    """
    Gather marginal and capital costs, p_nom_max for European countries
    from data from 
    'From hot rock to useful energy: A global estimate of enhanced
    geothermal systems potential' ~ Aghahosseini, Breyer

    Disaggregates data to the provided shapefile
    
    Args:
        egs_path(pathlib.Path): dir with required files for egs data
        shape_path(pathlib.Path): path to shapefiles to which data is disagreggated
    """

    shapes = gpd.read_file(respath / shape_path)
    times = pd.date_range('2015-01-01', '2055-01-01', freq='5y')

    cost_cutoffs = ["150", "100", "50"]
    databundle = dict()
    
    for cutoff in cost_cutoffs:
        inner = dict()
        inner['sus_potential'] = pd.DataFrame(index=times, columns=shapes['name'])
        inner['marginal_cost'] = deepcopy(inner['sus_potential'])
        inner['capital_cost'] = deepcopy(inner['sus_potential'])
        databundle[cutoff] = inner

    shapes['area'] = shapes.geometry.apply(lambda geom: geom.area)
    shapes['country'] = shapes['name'].apply(
        lambda name: pycountry.countries.get(alpha_2=name[:2]).name)

    areas = pd.DataFrame(shapes.groupby('country').sum()['area'])
    
    def get_overlap(country_row, shape_row):
        if shape_row.country != country_row.name:
            return 0.
        else:
            return shape_row.geometry.area / country_row.area

    for i, shape_row in shapes.iterrows():
        areas[shape_row['name']] = areas.apply(
            lambda country_row: get_overlap(country_row, shape_row),
            axis=1
        )

    country_shares = areas.drop(columns=['area'])
    assigner = np.ceil(country_shares)

    cutoff_slices = [slice(18,19), slice(8,18), slice(0,8)]

    # print("Potential data is not correct yet!")
    for cutoff, cutoff_slice in zip(cost_cutoffs, cutoff_slices):
        for i, time in enumerate(times):

            potential = pd.read_excel(egs_path / 'mmc3.xlsx', sheet_name=(i+1), index_col=0)
            potential = potential[[col for col in potential.columns if 'Power' in col]]

            potential = potential.loc["Afghanistan":"Zimbabwe"]
            potential = potential.rename(index=countryname_mapper)     

            potential = potential.loc[areas.index]
            potential = potential[potential.columns[cutoff_slice]]
            potential = potential.sum(axis=1)

            potential = country_shares.transpose() @ potential

            databundle[cutoff]['sus_potential'].loc[time] = potential
    
    # loading capex and opex at difference LCOE cutoffs
    cutoff_skiprows = [1, 44, 87]
    capex_usecols = slice(1, 9)
    opex_usecols = slice(10, 18)

    for cutoff, skiprows in zip(cost_cutoffs, cutoff_skiprows):
        prices = pd.read_excel(egs_path / "Geothermal_CapexOpex_Europe.xlsx",
                            sheet_name=1,
                            index_col=0,
                            skiprows=skiprows, 
                            )

        prices = prices.iloc[:38]
        prices = prices.rename(index=countryname_mapper) 
        prices = prices.loc[areas.index]

        capex = prices[prices.columns[capex_usecols]]
        opex = prices[prices.columns[opex_usecols]]

        opex.columns = times
        capex.columns = times

        for col in capex.columns:

            capex_by_shape = assigner.transpose() @ capex[col] 
            opex_by_shape = assigner.transpose() @ opex[col] 

            databundle[cutoff]['capital_cost'].loc[col] = capex_by_shape
            databundle[cutoff]['marginal_cost'].loc[col] = opex_by_shape

    print(databundle)

    return capex

shape_path = 'regions_onshore_elec_s_256.geojson'
capex = load_egs_data(egspath, shape_path)

{'150': {'sus_potential': name       AL1 0      AT1 0      AT1 1      AT1 2      AT1 3     AT1 4  \
2015-12-31   0.0  79.482767   30.41658  71.250491  47.275626   8.16525   
2020-12-31   0.0  79.482767   30.41658  71.250491  47.275626   8.16525   
2025-12-31   0.0  66.489253  25.444203  59.602755  39.547202  6.830428   
2030-12-31   0.0  66.489253  25.444203  59.602755  39.547202  6.830428   
2035-12-31   0.0  37.116515  14.203802  33.272242  22.076565  3.812973   
2040-12-31   0.0  37.116515  14.203802  33.272242  22.076565  3.812973   
2045-12-31   0.0  29.394954  11.248903  26.350428  17.483851  3.019738   
2050-12-31   0.0  22.292118    8.53078  19.983254  13.259149  2.290065   

name            BA1 0 BE1 0 BE1 1 BE1 2  ...      SE2 3      SE2 4      SE2 5  \
2015-12-31  60.461607   0.0   0.0   0.0  ...  15.981102  42.049827  50.181075   
2020-12-31  60.461607   0.0   0.0   0.0  ...  15.981102  42.049827  50.181075   
2025-12-31  60.461607   0.0   0.0   0.0  ...  15.981102  42.0498

In [27]:
shapes = gpd.read_file(respath / 'regions_onshore_elec_s_256.geojson')

country
Albania                    3.035618
Austria                    9.538171
Belgium                    3.897940
Bosnia and Herzegovina     5.832809
Bulgaria                  12.406771
Croatia                    5.894836
Czechia                    9.820071
Denmark                    5.649914
Estonia                    7.040807
Finland                   70.282824
France                    63.372533
Germany                   45.735335
Greece                    12.394470
Hungary                   11.056930
Ireland                    9.379799
Italy                     33.952535
Latvia                     9.504501
Lithuania                  9.169161
Luxembourg                 0.381974
Montenegro                 1.506044
Netherlands                4.805887
North Macedonia            2.741924
Norway                    49.529458
Poland                    41.147309
Portugal                   9.234535
Romania                   27.385253
Serbia                     9.929077
Slovakia            

In [8]:
from copy import deepcopy
countries = dict()

for i, row in fewshapes.iterrows():
    
    country = pycountry.countries.get(alpha_2=row.iloc[0][:2]).name
    if country not in countries:
        countries[country] = row.geometry.area
    else:
        countries[country] += row.geometry.area

countries_from_few = deepcopy(countries)
print(countries_from_few)

{'Albania': 3.03561771293909, 'Austria': 9.538170703093671, 'Bosnia and Herzegovina': 5.832809152144971, 'Belgium': 3.8979404166669887, 'Bulgaria': 12.40677110630455, 'Switzerland': 4.927989733925709, 'Czechia': 9.820071185199247, 'Germany': 45.735334977024294, 'Denmark': 5.649914323904262, 'Estonia': 7.040806751584686, 'Spain': 52.847589488131234, 'Finland': 70.2828236230089, 'France': 63.37253320246997, 'United Kingdom': 32.668206876302236, 'Greece': 12.39447034340076, 'Croatia': 5.89483582971477, 'Hungary': 11.056930349396989, 'Ireland': 9.379798925040742, 'Italy': 33.95253508941712, 'Lithuania': 9.169160845873655, 'Luxembourg': 0.38197447724561456, 'Latvia': 9.504500866608616, 'Montenegro': 1.5060441593829648, 'North Macedonia': 2.741923578615155, 'Netherlands': 4.805887257681788, 'Norway': 49.52945796270644, 'Poland': 41.147308689577834, 'Portugal': 9.234534988527269, 'Romania': 27.385253087403502, 'Serbia': 9.929077367240938, 'Sweden': 78.32151345655203, 'Slovenia': 2.87582613281

In [112]:
countries = dict()

for i, row in manyshapes.iterrows():
    
    country = pycountry.countries.get(alpha_2=row.iloc[0][:2]).name
    if country not in countries:
        countries[country] = row.geometry.area
    else:
        countries[country] += row.geometry.area

countries_from_many = deepcopy(countries)
print(countries_from_many)

{'Albania': 3.03561771293909, 'Austria': 9.538170703093673, 'Bosnia and Herzegovina': 5.832809152144971, 'Belgium': 3.897940416666987, 'Bulgaria': 12.406771106304555, 'Switzerland': 4.928048148384589, 'Czechia': 9.820071185199243, 'Germany': 45.73533497702427, 'Denmark': 5.649914323904262, 'Estonia': 7.040806751584686, 'Spain': 52.84758948813129, 'Finland': 70.28282362300891, 'France': 63.372533202470024, 'United Kingdom': 32.668206876302236, 'Greece': 12.394470343400751, 'Croatia': 5.89483582971477, 'Hungary': 11.056930349396982, 'Ireland': 9.37979892504074, 'Italy': 33.95253508941713, 'Lithuania': 9.169160845873655, 'Luxembourg': 0.38197447724561456, 'Latvia': 9.504500866608616, 'Montenegro': 1.5060441593829648, 'North Macedonia': 2.741923578615155, 'Netherlands': 4.805887257681793, 'Norway': 49.52945796270642, 'Poland': 41.14730868957786, 'Portugal': 9.234534988527267, 'Romania': 27.3852530874035, 'Serbia': 9.929077367240934, 'Sweden': 78.32151345655217, 'Slovenia': 2.87582613281235

In [21]:
shares_df = pd.DataFrame({'areas': list(countries_from_few.values())},
                         index=list(countries_from_few))

manyshapes['country'] = manyshapes['name'].apply(
    lambda name: pycountry.countries.get(alpha_2=name[:2]).name)



shares_df.drop(columns=['areas'], axis=0, inplace=True)

assign_df = np.ceil(shares_df)

print(shares_df.sum(axis=1).head(3))
# print(assign_df.head())
print(assign_df.tail())

np.allclose(shares_df.sum(axis=1).to_numpy(), np.ones(len(shares_df)), atol=1e-2)

Albania                   1.0
Austria                   1.0
Bosnia and Herzegovina    1.0
dtype: float64
          AL1 0  AT1 0  AT1 1  AT1 2  AT1 3  AT1 4  BA1 0  BE1 0  BE1 1  \
Romania     0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
Serbia      0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
Sweden      0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
Slovenia    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
Slovakia    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   

          BE1 2  ...  SE2 3  SE2 4  SE2 5  SE2 6  SE2 7  SE2 8  SE2 9  SI1 0  \
Romania     0.0  ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
Serbia      0.0  ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
Sweden      0.0  ...    1.0    1.0    1.0    1.0    1.0    1.0    1.0    0.0   
Slovenia    0.0  ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0    1.0   
Slovakia    0.0  ...    0.0    0.0    0.0   

True

In [17]:
regions = fewshapes.set_index("name").rename_axis("Bus")
buses = regions.index
regions.head()

,geometry
Bus,
AL1 0,"LINESTRING (20.32207 39.91318, 20.39703 39.818..."
AT1 0,"LINESTRING (14.05003 46.48440, 13.98233 46.481..."
BA1 0,"LINESTRING (17.82716 42.85312, 17.81176 42.909..."
BE1 0,"LINESTRING (4.68192 50.08392, 4.67292 50.01638..."
BG1 0,"LINESTRING (23.94989 41.43758, 23.89480 41.464..."


In [33]:
import atlite

cutout = atlite.Cutout(os.path.join(
    os.getcwd(),
    'pypsa-eur',
    'cutouts',
    'europe-2013-sarah.nc'
))
excluder = atlite.ExclusionContainer(crs=3035, res=100)

In [35]:
avail = cutout.availabilitymatrix(regions, excluder)

Compute availability matrix: 100%|██████████| 37/37 [00:24<00:00,  1.54 gridcells/s]


In [46]:
avail

<xarray.DataArray (Bus: 37, y: 210, x: 285)>
array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
...
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])
Coordinates:
  * Bus      (Bus) object 'AL1 0' 'AT1 0' 'BA1 0' ... 'SE2 0' 'SI1 0' 'SK1 0'
  * y        (y) float64 33.0 33.2 33.4 33.6 33.8 ... 74.0 74.2 74.4 74.6 74.8
  * x        (x) float64 -16.4 -16.2 -16.0 -15.8 -15.6 ... 39.8 40.0 40.2 40.4

In [49]:
import xarray as xr

area = cutout.grid.to_crs(3035).area / 1e6
area = xr.DataArray(
    area.values.reshape(cutout.shape), [cutout.coords['y'], cutout.coordss['x']]
)

In [ ]:
area